### Import

In [3]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ----- ---------------------------------- 16.8/124.9 MB 86.2 MB/s eta 0:00:02
   ------------- ------------------------- 41.9/124.9 MB 102.8 MB/s eta 0:00:01
   --------------------- ----------------- 67.4/124.9 MB 108.1 MB/s eta 0:00:01
   ---------------------------- ---------- 92.3/124.9 MB 110.4 MB/s eta 0:00:01
   ----------------------------------- -- 117.2/124.9 MB 111.6 MB/s eta 0:00:01
   -------------------------------------- 124.9/124.9 MB 101.7 MB/s eta 0:00:00


In [30]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder

import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

### Data Load

In [31]:
train = pd.read_csv('./Data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./Data/test.csv').drop(columns=['ID'])

In [32]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [33]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [34]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [35]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [9]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [36]:
numeric_columns = [col for col in numeric_columns if col in X.columns]

X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

In [64]:
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif

# 1️⃣ 상수 Feature 제거
var_thresh = VarianceThreshold(threshold=0.0)
X_train_var_filtered = var_thresh.fit_transform(X_train_encoded)

# 2️⃣ Feature Selection 적용
selector = SelectKBest(f_classif, k=20)
X_train_selected = selector.fit_transform(X_train_var_filtered, y)

# 2️⃣ SMOTE 적용 (Feature Selection 후)
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_selected, y)

In [65]:
print("X_train_encoded:", X_train_encoded.shape)
print("y:", y.shape)
print("X_train_selected:", X_train_selected.shape)
print("X_train_resampled:", X_train_resampled.shape)
print("y_resampled:", y_resampled.shape)

X_train_encoded: (256351, 67)
y: (256351,)
X_train_selected: (256351, 20)
X_train_resampled: (380246, 67)
y_resampled: (380246,)


In [66]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

### Train

XGBoost

In [67]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "lambda": trial.suggest_float("lambda", 1, 10),
        "alpha": trial.suggest_float("alpha", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
    }

    model = xgb.XGBClassifier(
        **params,
        objective="binary:logistic",
        eval_metric="auc",
        use_label_encoder=False,
        verbosity=0,  # 경고 메시지 숨김
        random_state=42,
        n_jobs=-1,
        scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1])
    )

    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
    y_pred = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, y_pred)

    return auc


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=150, show_progress_bar=True)

[I 2025-02-16 01:27:29,706] A new study created in memory with name: no-name-c1c343b6-ea03-4f00-abd1-0f0252ff3e71
  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\vhehr\AppData\Local\Temp\ipykernel_29528\2214655158.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
Best trial: 0. Best value: 0.906383:   1%|          | 1/100 [00:14<24:09, 14.64s/it]C:\Users\vhehr\AppData\Local\Temp\ipykernel_29528\2214655158.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),


[I 2025-02-16 01:27:44,349] Trial 0 finished with value: 0.9063829345118375 and parameters: {'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.1678756889016412, 'subsample': 0.8071020557907851, 'colsample_bytree': 0.7332391139768633, 'gamma': 1.9926006552330549, 'lambda': 6.761133455136932, 'alpha': 0.23872499618251442, 'min_child_weight': 10}. Best is trial 0 with value: 0.9063829345118375.


Best trial: 1. Best value: 0.906917:   2%|▏         | 2/100 [00:22<17:51, 10.93s/it]C:\Users\vhehr\AppData\Local\Temp\ipykernel_29528\2214655158.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),


[I 2025-02-16 01:27:52,681] Trial 1 finished with value: 0.906917425601085 and parameters: {'n_estimators': 200, 'max_depth': 14, 'learning_rate': 0.06234382367643746, 'subsample': 0.8500205740199213, 'colsample_bytree': 0.682456051313469, 'gamma': 2.109104712366281, 'lambda': 5.363802198821464, 'alpha': 4.936502259684005, 'min_child_weight': 3}. Best is trial 1 with value: 0.906917425601085.


Best trial: 1. Best value: 0.906917:   3%|▎         | 3/100 [00:48<28:39, 17.73s/it]C:\Users\vhehr\AppData\Local\Temp\ipykernel_29528\2214655158.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),


...

[I 2025-02-16 02:03:09,038] Trial 97 finished with value: 0.9075661873373906 and parameters: {'n_estimators': 800, 'max_depth': 13, 'learning_rate': 0.011719695924630675, 'subsample': 0.7691260048941203, 'colsample_bytree': 0.5474365001706301, 'gamma': 1.684443566945334, 'lambda': 2.9382781300595164, 'alpha': 0.28313161056666897, 'min_child_weight': 6}. Best is trial 91 with value: 0.9082859238683847.


Best trial: 91. Best value: 0.908286:  99%|█████████▉| 99/100 [36:15<00:34, 34.37s/it]C:\Users\vhehr\AppData\Local\Temp\ipykernel_29528\2214655158.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),


[I 2025-02-16 02:03:44,742] Trial 98 finished with value: 0.907206314749328 and parameters: {'n_estimators': 850, 'max_depth': 11, 'learning_rate': 0.01267360281060405, 'subsample': 0.7291326974406264, 'colsample_bytree': 0.5226159497243503, 'gamma': 1.5456631099546525, 'lambda': 2.431034117117983, 'alpha': 0.6337364902124597, 'min_child_weight': 6}. Best is trial 91 with value: 0.9082859238683847.


Best trial: 91. Best value: 0.908286: 100%|██████████| 100/100 [36:53<00:00, 22.13s/it]

[I 2025-02-16 02:04:22,808] Trial 99 finished with value: 0.9076724591636441 and parameters: {'n_estimators': 900, 'max_depth': 12, 'learning_rate': 0.015282505220064399, 'subsample': 0.7055903164112534, 'colsample_bytree': 0.5325557988623328, 'gamma': 1.3258005979879934, 'lambda': 3.3636225485788467, 'alpha': 0.18672967618162783, 'min_child_weight': 5}. Best is trial 91 with value: 0.9082859238683847.


In [ ]:
import joblib

# Optuna 최적화 실행 후 최적의 study 저장
joblib.dump(study, "xgb_optuna_study.pkl")


['xgb_optuna_study.pkl']

In [49]:
import joblib

# 저장된 Optuna study 불러오기
study = joblib.load("xgb_optuna_study.pkl")

# 최적의 하이퍼파라미터 가져오기
best_params = study.best_params
print("Best Params:", best_params)

# 최적의 하이퍼파라미터로 모델 학습
model = xgb.XGBClassifier(**best_params, random_state=42, n_jobs=-1)
model.fit(X_train_encoded, y)


FileNotFoundError: [Errno 2] No such file or directory: 'xgb_optuna_study.pkl'

In [69]:
# 최적 하이퍼파라미터 가져오기
best_params = study.best_params

# 최적의 파라미터로 모델 재학습
model = xgb.XGBClassifier(
    **best_params,
    objective="binary:logistic",
    eval_metric="auc",
    use_label_encoder=False,
    verbosity=0,
    random_state=42,
    n_jobs=-1
)

In [70]:
model.fit(X_train_resampled, y_resampled)

XGBClassifier(alpha=0.01734993356678516, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5125729570251544, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=1.5197921362681555,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=1.8940631696982866,
              learning_rate=0.01825270924751515, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=14, max_leaves=None,
              min_child_weight=7, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=750, n_jobs=-1, ...)

In [71]:
from sklearn.metrics import accuracy_score, roc_auc_score

# 예측
y_train_pred = model.predict(X_train_encoded)
y_train_proba = model.predict_proba(X_train_encoded)[:, 1]  # ROC-AUC Score 계산용

# 평가
accuracy = accuracy_score(y, y_train_pred)
roc_auc = roc_auc_score(y, y_train_proba)

# 출력
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")

Accuracy: 0.7784
ROC-AUC Score: 0.8090


RandomForest (Hyperparameter tune)

In [52]:
model.save_model("xgboost_SMOTE_model.json")

### Predict

In [72]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [73]:
sample_submission = pd.read_csv('./Data/sample_submission.csv')
sample_submission['probability'] = pred_proba

In [75]:
sample_submission.to_csv('./XGBoost_SMOTE3_optimization_submit.csv', index=False)